In [23]:



############ 
# as of now this one is the final code without previous content 
import os
from openai import AzureOpenAI
import openai
import json

# Define a chat function using API
def chat(prompt, model, client):
    message_text = [{"role": "system", "content": ""}, {"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model,  # model = "deployment_name"
        messages=message_text,
        temperature=0,
        max_tokens=200,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return completion

# Data loader function
def load_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def save_json_to_file(data, file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

# def load_problems_and_solutions(problems_dir, solutions_dir):
#     problem_solution_pairs = []

#     for filename in os.listdir(problems_dir):
#         if filename.endswith('.json'):
#             problem_file_path = os.path.join(problems_dir, filename)
#             solution_file_path = os.path.join(solutions_dir, filename)
            
#             problem_data = load_json_from_file(problem_file_path)
#             solution_data = load_json_from_file(solution_file_path)
            
#             source_language = problem_data.get("source_language")
#             target_language = problem_data.get("target_language")
#             train_data = problem_data.get("train", [])
#             test_problems = problem_data.get("test", [])
#             test_solutions = solution_data.get("test", [])

#             min_len = min(len(test_problems), len(test_solutions))
#             if min_len > 0:
#                 problem_solution_pairs.append({
#                     "source_language": source_language,
#                     "target_language": target_language,
#                     "train": train_data,
#                     "test_problems": [' '.join(filter(None, tp)) for tp in test_problems[:min_len]],  # Concatenate non-empty elements
#                     "test_solutions": [' '.join(filter(None, ts)) for ts in test_solutions[:min_len]] 
#                 }
#                 ) # Concatenate non-empty elements
#                 # print("t_p:",test_problems)
#                 # print(">>>>>>>>>>>>")
#                 # print("^^^^:",test_solutions)
                

#     return problem_solution_pairs
        
# def load_problems_and_solutions(problems_dir, solutions_dir):
#     problem_solution_pairs = []

#     for filename in os.listdir(problems_dir):
#         if filename.endswith('.json'):
#             problem_file_path = os.path.join(problems_dir, filename)
#             solution_file_path = os.path.join(solutions_dir, filename)
            
#             problem_data = load_json_from_file(problem_file_path)
#             solution_data = load_json_from_file(solution_file_path)
            
#             source_language = problem_data.get("source_language")
#             target_language = problem_data.get("target_language")
#             train_data = problem_data.get("train", [])
#             test_problems = problem_data.get("test", [])
#             test_solutions = solution_data.get("test", [])

#             min_len = min(len(test_problems), len(test_solutions))
#             if min_len > 0:
#                 problem_solution_pairs.append({
#                     "source_language": source_language,
                    
                    
#                     "target_language": target_language,
                    
#                     "train": train_data,
#                     "test_problems": [' '.join(filter(None, tp[:-1])) for tp in test_problems[:min_len]],  # Concatenate non-empty elements excluding the last element (">" or "<")
#                     "test_solutions": [ts[1] for ts in test_solutions[:min_len] if len(ts) > 1]  # Extract only the solutions
#                 })
#             # print("t_p:",test_problems)
#             # print(">>>>>>>>>>>>")
#             # print("^^^^:",test_solutions)
#             # print("s_l:",source_language)
#             # print("o_l:",target_language)

#     return problem_solution_pairs
        
def load_problems_and_solutions(problems_dir, solutions_dir):
    problem_solution_pairs = []

    for filename in os.listdir(problems_dir):
        if filename.endswith('.json'):
            problem_file_path = os.path.join(problems_dir, filename)
            solution_file_path = os.path.join(solutions_dir, filename)
            
            problem_data = load_json_from_file(problem_file_path)
            solution_data = load_json_from_file(solution_file_path)
            
            source_language = problem_data.get("source_language")
            target_language = problem_data.get("target_language")
            train_data = problem_data.get("train", [])
            test_problems = problem_data.get("test", [])
            test_solutions = solution_data.get("test", [])

            min_len = min(len(test_problems), len(test_solutions))
            if min_len > 0:
                problem_solution_pairs.append({
                    "source_language": source_language,
                    "target_language": target_language,
                    "train": train_data,
                    "test_problems": [' '.join(filter(None, tp)) for tp in test_problems[:min_len]],  # Concatenate non-empty elements
                    "test_solutions": [' '.join(filter(None, ts)) for ts in test_solutions[:min_len]] 
                }
                ) # Concatenate non-empty elements
                # print("t_p:",test_problems)
                # print(">>>>>>>>>>>>")
                # print("^^^^:",test_solutions)
                

    return problem_solution_pairs

# Interactive GPT function
def iterative_gpt_interaction(train_data, test_problems, actual_solutions, gpt4_iterations, gpt35_iterations, client):
    train_examples = "\n".join([f"Source: {example[0]}\nTranslation: {example[1]}" for example in train_data])
    # in the above train examples i want to give all the trainig example as a train 
    # so in the above i used source and translation of each example which are mentioned in the particular quastion 

    attempted_solutions = []
    all_hints = []
    for test_problem, actual_solution in zip(test_problems, actual_solutions):
        initial_prompt = (f"Use the following training examples to understand the language translation:\n\n"
                          f"{train_examples}\n\nNow, translate the following sentence:\n\n{test_problem}\n\n"
                          "Provide the translation in a single sentence without extra information, symbols, or explanations.")

        response = chat(initial_prompt, "gpt35turbo", client)
        response_content = response.choices[0].message.content

        initial_prompt_for_gpt4 = (f"Here we are providing the solution for each problem, so that you can use these hints "
                                   f"to generate an optimized solution from gpt35turbo. Please use the solution part and generate "
                                   f"hints for the following problem:\n\nProblem: {test_problem}\n\n"
                                   f"Generated Solution: {response_content}\n\n"
                                   f"Actual Solution: {actual_solution}\n\nHints:")

#         initial_prompt_for_gpt4 = """Problem: {test_problem}

# Expected solution: {actual_solution}

# Student's solution: {response_content}

# You are a helpful teaching assistant. Look at the linguistics olympiad problem above, its expected solution, and a student's attempted solution.
# If the student's solution 
# """


        hints = chat(initial_prompt_for_gpt4, "gpt4", client)
        hints_content = hints.choices[0].message.content
        all_hints.append(f"The below all hints for this problem: {test_problem}")
        all_hints.append(f"Initial Hints: {hints_content}")

        for i in range(gpt4_iterations):

            # so for this we need to give a intail prompt to gpt35 with generated hints from 4 , and the intial prompt will be the  there with hints 
            # and this will  contains the problem in this prompts
            intial_prompt=(f"so please here we are providing the problem {test_problem}\n, so please read this carefully and provide translation for this problem \n. "
                           f" so for the above problem you can use this hints {hints_content}\n for the translation."
                           f"Refine the translation in a single sentence without extra information, symbols, or explanations."
                           f" please dont add extra words and symbols  in the final response which you are generating"
                           ) 

            
            refinement_prompt = (f"{response_content}\n\nHints:\n{hints_content}\n\n"
                                 "Refine the translation in a single sentence without extra information, symbols, or explanations.")
            refined_response = chat(intial_prompt, "gpt35turbo", client)
            response_content = refined_response.choices[0].message.content

            # difference_analysis_prompt = (f"Actual Solution: {actual_solution}\nGenerated Solution: {response_content}\n\n"
            #                               "Generate hints for the differences and how to improve the solution:"
            #                               f"help in identify the problem occures in {response_content} and also analyze where the error is occuring ")
            
            difference_analysis_prompt = f"""Actual Solution: {actual_solution}
            Revised Generated Solution: {response_content}\n\n
Given above is the revised solution generated by gpt35turbo. Compare its solution with the actual solution to identify where and why do error(s) occur. 
Accordingly, generate a hint that helps gpt35turbo improve its solution. Directly address gpt35turbo as if it is a student and you are a teacher. Though you are premitted to point out mistakes in the generated solution, make sure to not reveal any details of the actual solution in your hint. Also note that gpt35turbo does not have access to the actual solution.
"""

            hints = chat(difference_analysis_prompt, "gpt4", client)
            hints_content = hints.choices[0].message.content
            all_hints.append(f"Hint Iteration {i+1}: {hints_content}")

        all_hints.append("--------------------------------------------------------------------------------\n")


        attempted_solutions.append(response_content)

    return attempted_solutions, all_hints

def main():
    problems_dir = "/Users/ramnarayanchoudhary/Desktop/UGRIP/new_repo/ugrip24-ling/global_iol_analysis/BBB_all_puzzling_problems/data_public_data_dev"  # Update with the correct path
    solutions_dir = "/Users/ramnarayanchoudhary/Desktop/UGRIP/new_repo/ugrip24-ling/global_iol_analysis/BBB_all_puzzling_problems/data_public_reference_data_dev"  # Update with the correct path  
    output_dir = "output_directory_of_gpt_interactions"  # Update with the desired output directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    problem_solution_pairs = load_problems_and_solutions(problems_dir, solutions_dir)

    # Initialize the OpenAI client
    api_key = "037155e1b16a432fa836637370eca0e3"  # Replace with your actual API key
    client = AzureOpenAI(
        azure_endpoint="https://cullmsouthindia.openai.azure.com/",
        api_key=api_key,
        api_version="2024-02-15-preview"
    )

    results = {}

    for pair in problem_solution_pairs:
        source_language = pair["source_language"]
        target_language = pair["target_language"]
        train_data = pair["train"]
        test_problems = pair["test_problems"]
        test_solutions = pair["test_solutions"]

        final_responses = None  # Initialize final_responses to ensure it's defined
        all_hints = None  # Initialize hints to ensure it's defined

        if source_language=="dyirbal":
         final_responses, all_hints = iterative_gpt_interaction(train_data, test_problems, test_solutions, gpt4_iterations=1, gpt35_iterations=1, client=client)

        #final_responses, all_hints = iterative_gpt_interaction(train_data, test_problems, test_solutions, gpt4_iterations=1, gpt35_iterations=1, client=client)

        if final_responses:  # Only add to results if final_responses is set
            language_pair = f"{source_language}-{target_language}"
            print("s_l:",source_language)
            if language_pair not in results:
                results[language_pair] = []
            results[language_pair].append({
               # "train": train_data,
                "test_problems": test_problems,
                "test_solutions": test_solutions,
                "attempted_solutions": final_responses,
                "hints": all_hints
            })

    # Save results in language-specific files
    for language_pair, data in results.items():
        output_file_path = os.path.join(output_dir, f"{language_pair}.json")
        save_json_to_file(data, output_file_path)

if __name__ == "__main__":
    main()


s_l: dyirbal


In [17]:
import os
from openai import AzureOpenAI
import openai
import json

# Define a chat function using API
def chat(prompt, model, client):
    message_text = [{"role": "system", "content": ""}, {"role": "user", "content": prompt}]
    completion = client.chat.completions.create(
        model=model,  # model = "deployment_name"
        messages=message_text,
        temperature=0,
        max_tokens=200,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return completion

# Data loader function
def load_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def save_json_to_file(data, file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

def load_problems_and_solutions(problems_dir, solutions_dir):
    problem_solution_pairs = []

    for filename in os.listdir(problems_dir):
        if filename.endswith('.json'):
            problem_file_path = os.path.join(problems_dir, filename)
            solution_file_path = os.path.join(solutions_dir, filename)
            
            problem_data = load_json_from_file(problem_file_path)
            solution_data = load_json_from_file(solution_file_path)
            
            source_language = problem_data.get("source_language")
            target_language = problem_data.get("target_language")
            train_data = problem_data.get("train", [])
            test_problems = problem_data.get("test", [])
            test_solutions = solution_data.get("test", [])

            min_len = min(len(test_problems), len(test_solutions))
            if min_len > 0:
                problem_solution_pairs.append({
                    "source_language": source_language,
                    "target_language": target_language,
                    "train": train_data,
                    "test_problems": [' '.join(filter(None, tp[:-1])) for tp in test_problems[:min_len]],  # Concatenate non-empty elements excluding the last element (">" or "<")
                    "test_solutions": [ts[1] for ts in test_solutions[:min_len] if len(ts) > 1]  # Extract only the solutions
                })

    return problem_solution_pairs

# Interactive GPT function
def iterative_gpt_interaction(train_data, test_problems, actual_solutions, gpt4_iterations, gpt35_iterations, client):
    train_examples = "\n".join([f"Source: {example[0]}\nTranslation: {example[1]}" for example in train_data])

    attempted_solutions = []
    all_hints = []
    prev_response = ""  # To store the previous response

    for test_problem, actual_solution in zip(test_problems, actual_solutions):
        initial_prompt = (f"Use the following training examples to understand the language translation:\n\n"
                          f"{train_examples}\n\nNow, translate the following sentence:\n\n{test_problem}\n\n"
                          "Provide the translation in a single sentence without extra information, symbols, or explanations.")

        response = chat(initial_prompt, "gpt35turbo", client)
        response_content = response.choices[0].message.content

        initial_prompt_for_gpt4 = (f"Here we are providing the solution for each problem, so that you can use these hints "
                                   f"to generate an optimized solution from gpt35turbo. Please use the solution part and generate "
                                   f"hints for the following problem:\n\nProblem: {test_problem}\n\n"
                                   f"Generated Solution: {response_content}\n\n"
                                   f"Actual Solution: {actual_solution}\n\nHints:")

        hints = chat(initial_prompt_for_gpt4, "gpt4", client)
        hints_content = hints.choices[0].message.content
        all_hints.append(f"The below all hints for this problem: {test_problem}")
        all_hints.append(f"Initial Hints: {hints_content}")

        for i in range(gpt4_iterations):
            message_text = [
                {"role": "system", "content": prev_response},
                {"role": "user", "content": f"{response_content}\n\nHints:\n{hints_content}"}
            ]

            refined_response = chat(message_text, "gpt35turbo", client)
            response_content = refined_response.choices[0].message.content

            difference_analysis_prompt = (f"Actual Solution: {actual_solution}\nGenerated Solution: {response_content}\n\n"
                                          "Generate hints for the differences and how to improve the solution:"
                                          f"help in identify the problem occures in {response_content} and also analyze where the error is occuring ")
            hints = chat(difference_analysis_prompt, "gpt4", client)
            hints_content = hints.choices[0].message.content
            all_hints.append(f"Hint Iteration {i+1}: {hints_content}")

            prev_response = response_content

        all_hints.append("--------------------------------------------------------------------------------\n")
        attempted_solutions.append(response_content)

    return attempted_solutions, all_hints

def main():
    problems_dir = "/Users/ramnarayanchoudhary/Desktop/UGRIP/new_repo/ugrip24-ling/global_iol_analysis/BBB_all_puzzling_problems/data_public_data_dev"  # Update with the correct path
    solutions_dir = "/Users/ramnarayanchoudhary/Desktop/UGRIP/new_repo/ugrip24-ling/global_iol_analysis/BBB_all_puzzling_problems/data_public_reference_data_dev"  # Update with the correct path  
    output_dir = "output_directory_of_gpt_interactions"  # Update with the desired output directory
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    problem_solution_pairs = load_problems_and_solutions(problems_dir, solutions_dir)

    # Initialize the OpenAI client
    api_key = "037155e1b16a432fa836637370eca0e3"  # Replace with your actual API key
    client = AzureOpenAI(
        azure_endpoint="https://cullmsouthindia.openai.azure.com/",
        api_key=api_key,
        api_version="2024-02-15-preview"
    )

    results = {}

    for pair in problem_solution_pairs:
        source_language = pair["source_language"]
        target_language = pair["target_language"]
        train_data = pair["train"]
        test_problems = pair["test_problems"]
        test_solutions = pair["test_solutions"]

        final_responses = None  # Initialize final_responses to ensure it's defined
        all_hints = None  # Initialize hints to ensure it's defined
        if source_language=="euskara":
         final_responses, all_hints = iterative_gpt_interaction(train_data, test_problems, test_solutions, gpt4_iterations=1, gpt35_iterations=1, client=client)

        if final_responses:  # Only add to results if final_responses is set
            language_pair = f"{source_language}-{target_language}"
            print("s_l:",source_language)
            if language_pair not in results:
                results[language_pair] = []
            results[language_pair].append({
                "test_problems": test_problems,
                "test_solutions": test_solutions,
                "attempted_solutions": final_responses,
                "hints": all_hints
            })

    # Save results in language-specific files
    for language_pair, data in results.items():
        output_file_path = os.path.join(output_dir, f"{language_pair}.json")
        save_json_to_file(data, output_file_path)

if __name__ == "__main__":
    main()
# import os
# from openai import AzureOpenAI
# import openai
# import json

# # Define a chat function using API
# def chat(message_text, model, client):
#     completion = client.chat.completions.create(
#         model=model,  # model = "deployment_name"
#         messages=message_text,
#         temperature=0,
#         max_tokens=200,
#         top_p=0.95,
#         frequency_penalty=0,
#         presence_penalty=0,
#         stop=None
#     )
#     return completion

# # Data loader function
# def load_json_from_file(file_path):
#     with open(file_path, 'r') as file:
#         return json.load(file)

# def save_json_to_file(data, file_path):
#     os.makedirs(os.path.dirname(file_path), exist_ok=True)
#     with open(file_path, 'w') as file:
#         json.dump(data, file, indent=4)

# def load_problems_and_solutions(problems_dir, solutions_dir):
#     problem_solution_pairs = []

#     for filename in os.listdir(problems_dir):
#         if filename.endswith('.json'):
#             problem_file_path = os.path.join(problems_dir, filename)
#             solution_file_path = os.path.join(solutions_dir, filename)
            
#             problem_data = load_json_from_file(problem_file_path)
#             solution_data = load_json_from_file(solution_file_path)
            
#             source_language = problem_data.get("source_language")
#             target_language = problem_data.get("target_language")
#             train_data = problem_data.get("train", [])
#             test_problems = problem_data.get("test", [])
#             test_solutions = solution_data.get("test", [])

#             min_len = min(len(test_problems), len(test_solutions))
#             if min_len > 0:
#                 problem_solution_pairs.append({
#                     "source_language": source_language,
#                     "target_language": target_language,
#                     "train": train_data,
#                     "test_problems": [' '.join(filter(None, tp[:-1])) for tp in test_problems[:min_len]],  # Concatenate non-empty elements excluding the last element (">" or "<")
#                     "test_solutions": [ts[1] for ts in test_solutions[:min_len] if len(ts) > 1]  # Extract only the solutions
#                 })

#     return problem_solution_pairs

# # Interactive GPT function
# def iterative_gpt_interaction(train_data, test_problems, actual_solutions, gpt4_iterations, gpt35_iterations, client):
#     train_examples = "\n".join([f"Source: {example[0]}\nTranslation: {example[1]}" for example in train_data])

#     attempted_solutions = []
#     all_hints = []
#     prev_response = ""  # To store the previous response

#     for test_problem, actual_solution in zip(test_problems, actual_solutions):
#         initial_prompt = (f"Use the following training examples to understand the language translation:\n\n"
#                           f"{train_examples}\n\nNow, translate the following sentence:\n\n{test_problem}\n\n"
#                           "Provide the translation in a single sentence without extra information, symbols, or explanations.")

#         response = chat([{"role": "system", "content": prev_response}, {"role": "user", "content": initial_prompt}], "gpt35turbo", client)
#         response_content = response.choices[0].message.content.strip()

#         initial_prompt_for_gpt4 = (f"Here we are providing the solution for each problem, so that you can use these hints "
#                                    f"to generate an optimized solution from gpt35turbo. Please use the solution part and generate "
#                                    f"hints for the following problem:\n\nProblem: {test_problem}\n\n"
#                                    f"Generated Solution: {response_content}\n\n"
#                                    f"Actual Solution: {actual_solution}\n\nHints:")

#         hints = chat([{"role": "system", "content": prev_response}, {"role": "user", "content": initial_prompt_for_gpt4}], "gpt4", client)
#         hints_content = hints.choices[0].message.content.strip()
#         all_hints.append(f"The below all hints for this problem: {test_problem}")
#         all_hints.append(f"Initial Hints: {hints_content}")

#         for i in range(gpt4_iterations):
#             message_text = [
#                 {"role": "system", "content": prev_response},
#                 {"role": "user", "content": f"{response_content}\n\nHints:\n{hints_content}"}
#             ]

#             refined_response = chat(message_text, "gpt35turbo", client)
#             response_content = refined_response.choices[0].message.content.strip()

#             difference_analysis_prompt = (f"Actual Solution: {actual_solution}\nGenerated Solution: {response_content}\n\n"
#                                           "Generate hints for the differences and how to improve the solution:"
#                                           f"help in identify the problem occures in {response_content} and also analyze where the error is occuring ")
#             hints = chat([{"role": "system", "content": prev_response}, {"role": "user", "content": difference_analysis_prompt}], "gpt4", client)
#             hints_content = hints.choices[0].message.content.strip()
#             all_hints.append(f"Hint Iteration {i+1}: {hints_content}")

#             prev_response = response_content

#         all_hints.append("--------------------------------------------------------------------------------\n")
#         attempted_solutions.append(response_content)

#     return attempted_solutions, all_hints

# def main():
#     problems_dir = "/Users/ramnarayanchoudhary/Desktop/UGRIP/new_repo/ugrip24-ling/global_iol_analysis/BBB_all_puzzling_problems/data_public_data_dev"  # Update with the correct path
#     solutions_dir = "/Users/ramnarayanchoudhary/Desktop/UGRIP/new_repo/ugrip24-ling/global_iol_analysis/BBB_all_puzzling_problems/data_public_reference_data_dev"  # Update with the correct path  
#     output_dir = "output_directory_of_gpt_interactions"  # Update with the desired output directory
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     problem_solution_pairs = load_problems_and_solutions(problems_dir, solutions_dir)

#     # Initialize the OpenAI client
#     api_key = "037155e1b16a432fa836637370eca0e3"  # Replace with your actual API key
#     client = AzureOpenAI(
#         azure_endpoint="https://cullmsouthindia.openai.azure.com/",
#         api_key=api_key,
#         api_version="2024-02-15-preview"
#     )

#     results = {}

#     for pair in problem_solution_pairs:
#         source_language = pair["source_language"]
#         target_language = pair["target_language"]
#         train_data = pair["train"]
#         test_problems = pair["test_problems"]
#         test_solutions = pair["test_solutions"]

#         final_responses = None  # Initialize final_responses to ensure it's defined
#         all_hints = None  # Initialize hints to ensure it's defined
#         if source_language=="euskara":
#          final_responses, all_hints = iterative_gpt_interaction(train_data, test_problems, test_solutions, gpt4_iterations=1, gpt35_iterations=1, client=client)

#         if final_responses:  # Only add to results if final_responses is set
#             language_pair = f"{source_language}-{target_language}"
#             print("s_l:",source_language)
#             if language_pair not in results:
#                 results[language_pair] = []
#             results[language_pair].append({
#                 "test_problems": test_problems,
#                 "test_solutions": test_solutions,
#                 "attempted_solutions": final_responses,
#                 "hints": all_hints
#             })

#     # Save results in language-specific files
#     for language_pair, data in results.items():
#         output_file_path = os.path.join(output_dir, f"{language_pair}.json")
#         save_json_to_file(data, output_file_path)

# if __name__ == "__main__":
#     main()


BadRequestError: Error code: 400 - {'error': {'message': "'$.messages[1].content' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}